# 4편: GAN으로 이미지 생성하기

이 노트북에서는 다음을 학습합니다:
- 분류(Discriminative) 모델과 생성(Generative) 모델의 차이
- GAN(Generative Adversarial Network)의 구조와 적대적 학습 원리
- BatchNorm을 활용한 Generator와 Dropout을 활용한 Discriminator 구현
- FashionMNIST 데이터셋으로 패션 아이템 이미지를 생성하는 GAN 학습
- 잠재 공간(Latent Space) 보간을 통한 이미지 탐험

## 1. 생성 모델이란?

머신러닝 모델은 크게 두 가지로 나뉩니다:

| 구분 | 분류(Discriminative) 모델 | 생성(Generative) 모델 |
|---|---|---|
| 목표 | 입력 데이터의 **클래스를 예측** | 학습 데이터와 유사한 **새로운 데이터 생성** |
| 학습 대상 | 결정 경계 $P(y|x)$ | 데이터 분포 $P(x)$ |
| 입력 → 출력 | 이미지 → 레이블 | 랜덤 노이즈 → 이미지 |
| 예시 | CNN 분류기, SVM | GAN, VAE, Diffusion Model |

1~3편에서 다룬 Linear, FC NN, CNN은 모두 분류 모델이었습니다.  
이번 4편에서는 **랜덤 노이즈로부터 새로운 이미지를 만들어내는** 생성 모델을 구현합니다.

## 2. GAN의 구조

GAN(Generative Adversarial Network)은 2014년 Ian Goodfellow가 제안한 생성 모델로,  
두 개의 신경망이 서로 **적대적(adversarial)**으로 경쟁하며 학습합니다.

### Generator (생성자) — 위조범
- 랜덤 노이즈 벡터 $z$를 입력받아 가짜 이미지를 생성합니다.
- 목표: Discriminator가 진짜라고 착각할 만큼 정교한 이미지를 만드는 것

### Discriminator (판별자) — 감정사
- 이미지를 입력받아 진짜(1)인지 가짜(0)인지 판별합니다.
- 목표: Generator가 만든 가짜 이미지를 정확히 가려내는 것

### 적대적 학습
```
랜덤 노이즈 z → [Generator] → 가짜 이미지 → [Discriminator] → 진짜/가짜 확률
                                진짜 이미지 → [Discriminator] → 진짜/가짜 확률
```

학습이 충분히 진행되면 Generator는 진짜와 구별할 수 없는 이미지를 생성하게 되고,  
Discriminator는 진짜와 가짜를 반반(확률 0.5)으로 예측하게 됩니다.  
이 상태를 **내시 균형(Nash Equilibrium)**이라고 합니다.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import numpy as np
from matplotlib import pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 디바이스: {device}")

In [ ]:
# 데이터 준비: [-1, 1] 범위로 정규화 (Tanh 출력과 맞추기 위해)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

train_dataset = dsets.FashionMNIST(
    root="data/", train=True, transform=transform, download=True
)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=64, shuffle=True, drop_last=True
)

print(f"학습 데이터: {len(train_dataset):,}장")
print(f"배치 수: {len(train_loader)} (배치 크기: 64)")
print(f"클래스: {train_dataset.classes}")

In [ ]:
def show_images(images, num_images=16, figsize=(8, 8)):
    """이미지 그리드 출력 (정규화 복원)"""
    images = images.cpu().detach()
    images = (images + 1) / 2  # [-1,1] -> [0,1]
    images = images.view(-1, 1, 28, 28)

    grid_size = int(num_images ** 0.5)
    fig, axes = plt.subplots(grid_size, grid_size, figsize=figsize)
    for i, ax in enumerate(axes.flat):
        if i < num_images:
            ax.imshow(images[i].squeeze(), cmap="gray")
        ax.axis("off")
    plt.tight_layout()
    plt.show()

## 3. 모델 구현

### Generator — 확장형 구조 + BatchNorm

Generator는 저차원 노이즈 벡터(64차원)를 점점 더 큰 차원으로 확장하여 이미지(784차원)를 생성합니다.

```
노이즈(64) → 128 → 256 → 512 → 784(=28×28)
```

각 Linear 층 뒤에 **BatchNorm1d**를 사용합니다. BatchNorm은 각 층의 출력을 정규화하여  
학습을 안정화하고 수렴 속도를 높이는 역할을 합니다.

### Discriminator — 축소형 구조 + Dropout

Discriminator는 이미지(784차원)를 점점 축소하며 진짜/가짜를 판별합니다.

```
이미지(784) → 512 → 256 → 1(확률)
```

**Dropout(0.3)**을 사용하여 Discriminator가 너무 강해지는 것을 방지합니다.  
Discriminator가 지나치게 강하면 Generator가 학습 신호를 받지 못해 학습이 실패할 수 있습니다.

In [ ]:
d_noise = 64


def make_noise(batch_size, d_noise=64):
    """잠재 공간에서 랜덤 노이즈 벡터 생성"""
    return torch.randn(batch_size, d_noise, device=device)


# Generator: 확장형 구조 (64 → 128 → 256 → 512 → 784)
G = nn.Sequential(
    nn.Linear(d_noise, 128),
    nn.BatchNorm1d(128),
    nn.LeakyReLU(0.2),
    nn.Linear(128, 256),
    nn.BatchNorm1d(256),
    nn.LeakyReLU(0.2),
    nn.Linear(256, 512),
    nn.BatchNorm1d(512),
    nn.LeakyReLU(0.2),
    nn.Linear(512, 28 * 28),
    nn.Tanh(),
).to(device)

# Discriminator: 축소형 구조 (784 → 512 → 256 → 1)
D = nn.Sequential(
    nn.Linear(28 * 28, 512),
    nn.LeakyReLU(0.2),
    nn.Dropout(0.3),
    nn.Linear(512, 256),
    nn.LeakyReLU(0.2),
    nn.Dropout(0.3),
    nn.Linear(256, 1),
    nn.Sigmoid(),
).to(device)

print("=== Generator (확장형 + BatchNorm) ===")
print(f"파라미터 수: {sum(p.numel() for p in G.parameters()):,}")
print(f"\n=== Discriminator (축소형 + Dropout) ===")
print(f"파라미터 수: {sum(p.numel() for p in D.parameters()):,}")

In [ ]:
# 학습 전 Generator 출력 확인 (초기화 직후 — 의미 없는 노이즈)
G.eval()
with torch.no_grad():
    z = make_noise(1)
    fake_img = G(z).view(28, 28).cpu()

print("학습 전 Generator 출력 (랜덤 노이즈에서 생성):")
plt.figure(figsize=(3, 3))
plt.imshow((fake_img + 1) / 2, cmap="gray")
plt.axis("off")
plt.show()

# Discriminator 판별 결과
D.eval()
with torch.no_grad():
    z = make_noise(5)
    fake_batch = G(z)
    probs = D(fake_batch)

print(f"Discriminator 판별 결과 (학습 전): {[f'{p:.4f}' for p in probs.squeeze().tolist()]}")
print("  -> 약 0.5 근처 (학습 전이라 진짜/가짜 구별 못함)")

## 4. GAN 학습

매 배치마다 Discriminator와 Generator를 번갈아 학습합니다.

### Discriminator 학습
- 진짜 이미지 → D → **0.9에 가깝게** (Label Smoothing 적용)
- 가짜 이미지 → D → **0에 가깝게**
- Label Smoothing: 진짜 레이블을 1.0 대신 0.9로 설정하여 Discriminator의 과신을 방지

### Generator 학습
- 가짜 이미지 → D → **1에 가깝게** (Discriminator를 속이는 방향으로)

### 손실 함수: BCELoss
Binary Cross Entropy Loss를 사용합니다:
$$\text{BCE}(p, y) = -[y \cdot \log(p) + (1-y) \cdot \log(1-p)]$$

In [ ]:
criterion = nn.BCELoss()


def train_discriminator(real_images, optimizer_d):
    """Discriminator 1스텝 학습: 진짜는 0.9, 가짜는 0으로 판별"""
    optimizer_d.zero_grad()
    batch_size = real_images.size(0)

    real_labels = torch.full((batch_size, 1), 0.9, device=device)  # label smoothing
    fake_labels = torch.zeros(batch_size, 1, device=device)

    # 진짜 이미지 판별
    pred_real = D(real_images.view(-1, 28 * 28))
    loss_real = criterion(pred_real, real_labels)

    # 가짜 이미지 판별
    noise = make_noise(batch_size)
    fake_images = G(noise)
    pred_fake = D(fake_images.detach())  # Generator 그래프 분리
    loss_fake = criterion(pred_fake, fake_labels)

    loss_d = loss_real + loss_fake
    loss_d.backward()
    optimizer_d.step()

    return loss_d.item(), pred_real.mean().item(), pred_fake.mean().item()


def train_generator(batch_size, optimizer_g):
    """Generator 1스텝 학습: 가짜 이미지를 진짜(1)로 판별하게 만들기"""
    optimizer_g.zero_grad()

    noise = make_noise(batch_size)
    fake_images = G(noise)
    pred_fake = D(fake_images)

    real_labels = torch.ones(batch_size, 1, device=device)
    loss_g = criterion(pred_fake, real_labels)

    loss_g.backward()
    optimizer_g.step()

    return loss_g.item()

In [ ]:
# 가중치 초기화: Kaiming Normal (He 초기화)
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)


G.apply(init_weights)
D.apply(init_weights)

# Optimizer: Adam with beta1=0.5 (GAN 학습에 효과적)
opt_g = optim.Adam(G.parameters(), lr=0.0001, betas=(0.5, 0.999))
opt_d = optim.Adam(D.parameters(), lr=0.0001, betas=(0.5, 0.999))

# 고정 노이즈: 에포크마다 같은 노이즈로 생성하여 진행 상황 추적
fixed_noise = make_noise(16)

# 학습 기록
loss_g_history = []
loss_d_history = []
p_real_history = []
p_fake_history = []

total_epochs = 100
print(f"GAN 학습 시작 ({total_epochs} 에포크)...\n")

for epoch in range(total_epochs):
    G.train()
    D.train()
    epoch_loss_g, epoch_loss_d = 0, 0
    epoch_p_real, epoch_p_fake = 0, 0
    num_batches = 0

    for images, _ in train_loader:
        images = images.to(device)
        bs = images.size(0)

        loss_d, p_r, p_f = train_discriminator(images, opt_d)
        loss_g = train_generator(bs, opt_g)

        epoch_loss_d += loss_d
        epoch_loss_g += loss_g
        epoch_p_real += p_r
        epoch_p_fake += p_f
        num_batches += 1

    # 에포크 평균
    loss_g_history.append(epoch_loss_g / num_batches)
    loss_d_history.append(epoch_loss_d / num_batches)
    p_real_history.append(epoch_p_real / num_batches)
    p_fake_history.append(epoch_p_fake / num_batches)

    if (epoch + 1) % 20 == 0:
        print(
            f"Epoch {epoch+1:3d}/{total_epochs} | "
            f"Loss_D: {loss_d_history[-1]:.4f} | Loss_G: {loss_g_history[-1]:.4f} | "
            f"p_real: {p_real_history[-1]:.4f} | p_fake: {p_fake_history[-1]:.4f}"
        )
        G.eval()
        with torch.no_grad():
            generated = G(fixed_noise)
        show_images(generated)

## 5. 학습 결과 분석

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 왼쪽: 손실 곡선
ax1.plot(loss_d_history, label="Discriminator Loss", alpha=0.8)
ax1.plot(loss_g_history, label="Generator Loss", alpha=0.8)
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss")
ax1.set_title("Loss Curves")
ax1.legend()
ax1.grid(True, alpha=0.3)

# 오른쪽: 판별 확률 수렴
ax2.plot(p_real_history, label="p_real (진짜 -> 진짜)", alpha=0.8)
ax2.plot(p_fake_history, label="p_fake (가짜 -> 진짜)", alpha=0.8)
ax2.axhline(y=0.5, color="gray", linestyle="--", alpha=0.5, label="이상적 수렴점 (0.5)")
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Probability")
ax2.set_title("Discriminator Predictions")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("p_real과 p_fake가 모두 0.5 근처로 수렴하면 이상적인 상태입니다.")
print("-> Discriminator가 진짜/가짜를 구별하지 못한다는 의미")

In [ ]:
# 최종 생성 이미지 (고정 노이즈 사용)
G.eval()
with torch.no_grad():
    final_images = G(fixed_noise)

print("최종 Generator가 생성한 패션 아이템 이미지 (4x4):")
show_images(final_images)

## 6. 노이즈 공간 탐험

Generator는 64차원 노이즈 벡터를 입력받아 이미지를 생성합니다.  
이 노이즈 공간(잠재 공간, Latent Space)에서 **두 점 사이를 보간(interpolation)**하면,  
생성되는 이미지가 부드럽게 변화하는 것을 관찰할 수 있습니다.

이는 Generator가 단순히 이미지를 암기한 것이 아니라,  
잠재 공간의 **연속적인 표현**을 학습했다는 증거입니다.

In [ ]:
G.eval()
z1 = make_noise(1)
z2 = make_noise(1)

steps = 8
fig, axes = plt.subplots(1, steps, figsize=(16, 2))
for i, alpha in enumerate(np.linspace(0, 1, steps)):
    z = z1 * (1 - alpha) + z2 * alpha
    with torch.no_grad():
        img = G(z).view(28, 28).cpu()
    img = (img + 1) / 2  # [-1,1] -> [0,1]
    axes[i].imshow(img, cmap="gray")
    axes[i].set_title(f"\u03b1={alpha:.2f}", fontsize=9)
    axes[i].axis("off")
plt.suptitle("Latent Space Interpolation", fontsize=14)
plt.tight_layout()
plt.show()

## 정리

### GAN 핵심 개념

| 구성 요소 | 역할 | 목표 |
|---|---|---|
| Generator | 노이즈 → 가짜 이미지 | Discriminator를 속이기 |
| Discriminator | 이미지 → 진짜 확률 | Generator를 간파하기 |
| 적대적 학습 | 두 모델의 경쟁 | 균형점(Nash Equilibrium) 도달 |

### 이 노트북에서 사용한 기법

| 기법 | 적용 대상 | 효과 |
|---|---|---|
| BatchNorm1d | Generator | 학습 안정화, 수렴 속도 향상 |
| Dropout(0.3) | Discriminator | 과적합 방지, D가 너무 강해지는 것 억제 |
| Label Smoothing (0.9) | D 학습 시 | D의 과신 방지 |
| Kaiming 초기화 | 전체 | LeakyReLU에 적합한 가중치 초기화 |
| Adam(beta1=0.5) | 전체 | GAN 학습에 효과적인 모멘텀 설정 |

### GAN 학습의 어려움
- **모드 붕괴(Mode Collapse)**: Generator가 다양성 없이 한두 가지 이미지만 반복 생성
- **학습 불안정**: Generator와 Discriminator의 학습 속도 균형 맞추기가 어려움
- **평가 기준 부재**: 생성 이미지의 품질을 객관적으로 측정하기 어려움 (FID, IS 등의 지표 사용)

### 딥러닝 기초 시리즈 요약

| 편 | 모델 | 핵심 개념 |
|---|---|---|
| 1편 | Linear | 이미지 = 숫자 벡터, 학습 = weight 업데이트 |
| 2편 | FC NN | 은닉층을 쌓으면 표현력 증가, 파라미터 폭발 문제 |
| 3편 | CNN | 지역성 활용 → 적은 파라미터로 높은 성능 |
| 4편 | GAN | Generator vs Discriminator 경쟁 → 이미지 생성 |